(optional) resize the background image

In [3]:
import os
from PIL import Image

background_folder = '/home/dfrobot/yolo/traffic_sign/image_generate/background'
output_folder = '/home/dfrobot/yolo/traffic_sign/image_generate/resized_background'

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Resize the images
for filename in os.listdir(background_folder):
    if filename.endswith('.jpg') or filename.endswith('.jpeg'):
        image_path = os.path.join(background_folder, filename)
        output_path = os.path.join(output_folder, filename)
        
        # Open the image
        image = Image.open(image_path)
        
        # Resize the image to 640x640
        resized_image = image.resize((640, 640))
        
        # Save the resized image
        resized_image.save(output_path)

print('Images resized successfully.')

Images resized successfully.


生成处理后的前景图

In [1]:
import os
import random
from PIL import Image, ImageChops, ImageEnhance
import math
import numpy as np
from PIL import ImageFilter
import cv2

# 配置路径
root_dir = '/home/dfrobot/yolo/traffic/all_in_one/image/front_img'
background_folder = '/home/dfrobot/yolo/traffic/all_in_one/image/background'
# 每张前景图图片随机生成的数量
random_image_count = 20


# 获取根目录下的所有文件夹
folders = [f for f in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, f))]

# 创建对应的 cropped_image_folders 和 output_folders
cropped_image_folders = {}
output_folders = {}

for folder in folders:
    cropped_image_folder = os.path.join('cropped_image', folder)
    output_folder = os.path.join('output', folder)
    
    # 创建文件夹
    os.makedirs(cropped_image_folder, exist_ok=True)
    os.makedirs(output_folder, exist_ok=True)
    
    cropped_image_folders[folder] = cropped_image_folder
    output_folders[folder] = output_folder

# 剪裁透明部分
def trim(im):
    bg = Image.new(im.mode, im.size, (0, 0, 0, 0))
    diff = ImageChops.difference(im, bg)
    bbox = diff.getbbox()
    if bbox:
        return im.crop(bbox)
    else:
        return im

# 将图像变为320*320大小
def resize_image(image):
    return image.resize((320, 320))

# 向图像添加随机噪声
def add_noise(image):
    np_image = np.array(image)
    noise = np.random.randint(0, 1, (np_image.shape[0], np_image.shape[1], 4), dtype='uint8')
    np_image = np_image + noise
    return Image.fromarray(np.clip(np_image, 0, 255).astype('uint8'))

# 高斯模糊
def add_gaussian_blur(image):
    blur_radius = random.uniform(0, 1.5)
    return image.filter(ImageFilter.GaussianBlur(blur_radius))

# 添加运动模糊
def add_motion_blur(image):
    kernel_size = random.randint(5, 20)
    kernel = np.zeros((kernel_size, kernel_size))
    kernel[int((kernel_size - 1) / 2), :] = np.ones(kernel_size)
    kernel = kernel / kernel_size
    np_image = np.array(image)
    for i in range(3):
        np_image[:, :, i] = cv2.filter2D(np_image[:, :, i], -1, kernel)
    return Image.fromarray(np_image)

# 随机缩放图像
def random_scale(image):
    scale_factor = random.uniform(0.2, 1.4)
    new_size = (int(image.width * scale_factor), int(image.height * scale_factor))
    return image.resize(new_size, Image.Resampling.LANCZOS)

# 随机旋转图像
def random_rotate(image):
    angle = random.uniform(-15, 15)
    return image.rotate(angle, expand=True)

# 随机调整亮度
def random_brightness(image):
    enhancer = ImageEnhance.Brightness(image)
    factor = random.uniform(0.6, 1.3)
    return enhancer.enhance(factor)

# 随机改变图像的长宽比
def random_aspect_ratio(image):
    width_factor = random.uniform(0.8, 1)
    height_factor = random.uniform(0.9, 1.2)
    new_width = int(image.width * width_factor)
    new_height = int(image.height * height_factor)
    return image.resize((new_width, new_height), Image.Resampling.LANCZOS)

# 剪裁所有动物图像并保存
for image in folders:
    folder = os.path.join(root_dir, image)
    image_files = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith('.png')]
    for image_file in image_files:
        img = Image.open(image_file).convert('RGBA')
        for i in range(random_image_count):
            processed_img = resize_image(img)
            processed_img = add_noise(processed_img)
            processed_img = random_scale(processed_img)
            processed_img = random_rotate(processed_img)
            processed_img = random_brightness(processed_img)
            processed_img = add_motion_blur(processed_img)
            processed_img = random_aspect_ratio(processed_img)
            processed_img = trim(processed_img)
            base_name = os.path.basename(image_file)
            name, ext = os.path.splitext(base_name)
            new_name = f"{name}_{i}{ext}"
            processed_img.save(os.path.join(cropped_image_folders[image], new_name))

print('剪裁完成')

剪裁完成


随机合成前景图和背景图

In [3]:


# YOLO标注格式：class x_center y_center width height
def create_yolo_annotation(image_size, bbox, class_id):
    dw = 1. / image_size[0]
    dh = 1. / image_size[1]
    x_center = (bbox[0] + bbox[2]) / 2.0
    y_center = (bbox[1] + bbox[3]) / 2.0
    width = bbox[2] - bbox[0]
    height = bbox[3] - bbox[1]
    x_center *= dw
    y_center *= dh
    width *= dw
    height *= dh
    return f"{class_id} {x_center} {y_center} {width} {height}"

# 计算两个中心点之间的距离
def center_distance(bbox1, bbox2):
    x1_center = (bbox1[0] + bbox1[2]) / 2.0
    y1_center = (bbox1[1] + bbox1[3]) / 2.0
    x2_center = (bbox2[0] + bbox2[2]) / 2.0
    y2_center = (bbox2[1] + bbox2[3]) / 2.0
    return math.sqrt((x2_center - x1_center) ** 2 + (y2_center - y1_center) ** 2)

# 生成合成图像和标注
def generate_composite_image_and_annotation(background_img, animal_imgs, animal_classes, output_path, annotation_path):
    bg_width, bg_height = background_img.size
    annotations = []
    placed_bboxes = []

    for animal_img, class_id in zip(animal_imgs, animal_classes):
        animal_width, animal_height = animal_img.size
        max_x = bg_width - animal_width
        max_y = bg_height - animal_height
        if max_x < 0 or max_y < 0:
            print("Error: Background image is smaller than the front image.")
            continue

        for _ in range(100):  # 尝试最多100次以找到合适的位置
            x = random.randint(0, max_x)
            y = random.randint(0, max_y)
            bbox = (x, y, x + animal_width, y + animal_height)
            if all(center_distance(bbox, placed_bbox) > 150 for placed_bbox in placed_bboxes):
                break
        else:
            print("Warning: Could not find suitable position for front image.")
            continue

        # Paste the animal image onto the background
        background_img.paste(animal_img, (x, y), animal_img)

        # 记录已放置的bbox
        placed_bboxes.append(bbox)

        # 转换为YOLO标注格式
        annotation = create_yolo_annotation((bg_width, bg_height), bbox, class_id)
        annotations.append(annotation)

    # Convert to RGB mode before saving
    background_img = background_img.convert('RGB')
    background_img.save(output_path, 'JPEG')

    # Save the annotations
    with open(annotation_path, 'w') as f:
        for ann in annotations:
            f.write(ann + '\n')

# 读取背景图和剪裁后的动物图像
# 注意这里要改图片后缀
backgrounds = [os.path.join(background_folder, f) for f in os.listdir(background_folder) if f.endswith('.jpg')]
print(backgrounds)
cropped_animals = {animal: [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith('.png')] for animal, folder in cropped_animal_folders.items()}

# 生成合成图像和标注
for animal, animal_files in cropped_animals.items():
    for bg_file in backgrounds:
        background_img = Image.open(bg_file).convert('RGBA')
        
        for i in range(1, 2):  # 每个背景图生成5张不同组合的图像
            num_animals = random.randint(1, 2)
            selected_animals = []
            selected_classes = []
            for _ in range(num_animals):
                animal_file = random.choice(animal_files)
                selected_animals.append(Image.open(animal_file).convert('RGBA'))
                selected_classes.append(list(cropped_animals.keys()).index(animal))
            
            output_image_path = os.path.join(output_folders[animal], f'composite_{os.path.splitext(os.path.basename(bg_file))[0]}_{i}.jpg')
            output_annotation_path = os.path.join(output_folders[animal], f'composite_{os.path.splitext(os.path.basename(bg_file))[0]}_{i}.txt')
            generate_composite_image_and_annotation(background_img.copy(), selected_animals, selected_classes, output_image_path, output_annotation_path)

print("生成完毕！")

['/home/dfrobot/yolo/traffic/all_in_one/image/background/output222_frame_0218.jpg', '/home/dfrobot/yolo/traffic/all_in_one/image/background/12.jpg', '/home/dfrobot/yolo/traffic/all_in_one/image/background/304.jpg', '/home/dfrobot/yolo/traffic/all_in_one/image/background/output-0906_frame_0132.jpg', '/home/dfrobot/yolo/traffic/all_in_one/image/background/89.jpg', '/home/dfrobot/yolo/traffic/all_in_one/image/background/output-0906_frame_1092.jpg', '/home/dfrobot/yolo/traffic/all_in_one/image/background/output-0906_frame_0148.jpg', '/home/dfrobot/yolo/traffic/all_in_one/image/background/output-0906_frame_1964.jpg', '/home/dfrobot/yolo/traffic/all_in_one/image/background/output5_frame_0092.jpg', '/home/dfrobot/yolo/traffic/all_in_one/image/background/output-0906_frame_0612.jpg', '/home/dfrobot/yolo/traffic/all_in_one/image/background/output3_frame_0166.jpg', '/home/dfrobot/yolo/traffic/all_in_one/image/background/565.jpg', '/home/dfrobot/yolo/traffic/all_in_one/image/background/output-0906

（可选）合成单张图片中多分类数据

In [4]:
# 创建muti文件夹路径
muti_output_folder = 'output/muti_output'
if not os.path.exists(muti_output_folder):
    os.makedirs(muti_output_folder)

# 生成muti文件夹中的图像和标注
for bg_file in backgrounds:
    background_img = Image.open(bg_file).convert('RGBA')
    # 随机选择三种不同的水果
    selected_animal_types = random.sample(list(cropped_animals.keys()), 3)
    selected_animals = []
    selected_classes = []
    for animal_type in selected_animal_types:
        animal_file = random.choice(cropped_animals[animal_type])
        selected_animals.append(Image.open(animal_file).convert('RGBA'))
        selected_classes.append(list(cropped_animals.keys()).index(animal_type))
    
    # 生成输出路径
    output_image_path = os.path.join(muti_output_folder, f'muti_{os.path.splitext(os.path.basename(bg_file))[0]}.jpg')
    output_annotation_path = os.path.join(muti_output_folder, f'muti_{os.path.splitext(os.path.basename(bg_file))[0]}.txt')
    generate_composite_image_and_annotation(background_img, selected_animals, selected_classes, output_image_path, output_annotation_path)

print("muti文件夹中的图像和标注生成完毕！")

Error: Background image is smaller than the front image.
Error: Background image is smaller than the front image.
Error: Background image is smaller than the front image.
Error: Background image is smaller than the front image.
Error: Background image is smaller than the front image.
Error: Background image is smaller than the front image.
Error: Background image is smaller than the front image.
Error: Background image is smaller than the front image.
Error: Background image is smaller than the front image.
Error: Background image is smaller than the front image.
Error: Background image is smaller than the front image.
Error: Background image is smaller than the front image.
Error: Background image is smaller than the front image.
Error: Background image is smaller than the front image.
Error: Background image is smaller than the front image.
Error: Background image is smaller than the front image.
Error: Background image is smaller than the front image.
Error: Background image is smal